In [2]:
import pandas as pd

# Read the TSV file, skipping the commented header lines and combining multi-line entries
with open('satcat.tsv', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

entries = []
current = []
for line in lines:
    if not line or line.startswith('#'):
        continue
    # New entry starts when a line begins with an alphanumeric catalog ID (e.g., "S#####")
    if line[0].isalnum():
        if current:
            entries.append("\t".join(seg.strip() for seg in current))
        current = [line]
    else:
        current.append(line)
if current:
    entries.append("\t".join(seg.strip() for seg in current))

# Create DataFrame from parsed entries
cols = ["JCAT","Satcat","Launch_Tag","Piece","Type","Name","PLName","LDate","Parent",
        "SDate","Primary","DDate","Status","Dest","Owner","State","Manufacturer","Bus",
        "Motor","Mass","MassFlag","DryMass","DryFlag","TotMass","TotFlag","Length","LFlag",
        "Diameter","DFlag","Span","SpanFlag","Shape","ODate","Perigee","PF","Apogee","AF",
        "Inc","IF","OpOrbit","OQUAL","AltNames"]
df_all = pd.DataFrame([entry.split("\t") for entry in entries], columns=cols)

# Strip whitespace from all columns
df_all = df_all.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_all.shape
# Filter to payloads (Type starting with 'P')
df_payloads = df_all[df_all['Type'].str.startswith('P')].copy()

# Extract launch year from LDate (format e.g. "1957 Oct 4")
df_payloads['Year'] = df_payloads['LDate'].str[:4].astype(int)

# Map country codes in 'State' to standardized names
country_map = {
    'US': 'United States',
    'SU': 'USSR',             # Soviet Union (pre-1992)
    'RU': 'Russia',           # Russia (post-1991)
    'CN': 'China',
    'UK': 'United Kingdom',
    'J':  'Japan',
    'F':  'France',
    'IN': 'India',
    'D':  'Germany',
    'CA': 'Canada',
    'IL': 'Israel',
    'I':  'Italy',
    'BR': 'Brazil',
    'ROK': 'South Korea',     # Republic of Korea
    'IR': 'Iran',
    'NK': 'North Korea',
    'AUS': 'Australia',
    'UAE': 'United Arab Emirates',
    'RA': 'Argentina',
    'MEX': 'Mexico',
    'TR': 'Turkey',
    'RI': 'Indonesia'
}

df_payloads['Country'] = df_payloads['State'].map(country_map).fillna(df_payloads['State'])
# Combine USSR/Russia for analysis by replacing 'USSR' and 'Russia' with a single label where appropriate
df_payloads['CountryGroup'] = df_payloads['Country'].replace({'USSR': 'USSR/Russia', 'Russia': 'USSR/Russia'})
# Simplify orbit category codes to broad classes
def classify_orbit(oporbit):
    if pd.isna(oporbit) or oporbit == '' or oporbit == '-':
        return 'Unknown'
    code = oporbit.upper()
    if code in ['SO','TA']:
        return 'Suborbital'
    if 'LEO' in code:
        return 'LEO'
    if code.startswith('GEO') or code == 'GEO':
        return 'GEO'
    if code == 'MEO':
        return 'MEO'
    if code.startswith('HEO') or code == 'GTO':
        return 'Elliptical'
    if code in ['DSO','CLO','EEO','HCO','PCO','SSE']:
        return 'Deep Space'
    return 'Other'
df_payloads['OrbitClass'] = df_payloads['OpOrbit'].apply(classify_orbit)
# Determine if satellite is still in orbit (active) or not
df_payloads['InOrbit'] = df_payloads['Status'].str.startswith('O')
# Define keywords for mission type classification
comm_keywords = ["SATCOM", "INTELSAT", "TELSTAR", "COMSAT", "COMMUNICATION", "BROADCAST",
                 "DIRECTV", "IRIDIUM", "GLOBALSTAR", "STARLINK", "ONEWEB", "INMARSAT", "ARSAT",
                 "HISPASAT", "TURKSAT", "EXPRESS", "YAMAL", "ECHOSTAR", "JCSAT", "CHINASAT", "SES"]
mil_keywords  = ["COSMOS", "YAOGAN", "NAVSTAR", "GPS", "GLONASS", "SBIRS", "NROL", "USA ",
                 "OFEQ", "DSP ", "MILSTAR", "LACROSSE", "KH-", "SPYSAT", "RECON", "ZYA", "OKO"]
sci_keywords  = ["EXPLORER", "HUBBLE", "CHANDRA", "ISS", "SKYLAB", "TIANGONG", "SPUTNIK",
                 "VOYAGER", "PIONEER", "GALILEO", "VENERA", "MARS", "LUNA", "APOLLO", "SOHO",
                 "HST", "JAMES WEBB", "KEPLER", "ROSETTA", "LANDSAT", "TIROS", "NOAA", "GOES", "METEOR", "COS-B"]

def classify_mission(name, owner):
    text = f"{name} {owner}".upper()
    if any(kw in text for kw in comm_keywords):
        return 'Communications'
    if any(kw in text for kw in mil_keywords):
        return 'Military'
    if any(kw in text for kw in sci_keywords):
        return 'Scientific'
    return 'Other'

df_payloads['MissionType'] = df_payloads.apply(lambda row: classify_mission(row['Name'], row['Owner']), axis=1)


<ipython-input-2-5b725a59d675>:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_all = df_all.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [12]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use('ggplot')

# =========================
#  0) Preprocessing (apply only once)
# =========================
status_mapping = {
    'O': 'Active',
    'D': 'Decayed',
    'DK': 'Decayed',
    'DSA': 'Decayed',
    'DSO': 'Decayed',
    'N': 'Defunct',
    'R': 'Defunct',
    'R?': 'Defunct'
}
df_payloads['OperationalStatus'] = df_payloads['Status'].map(status_mapping)

# =========================
#  1) Define Interactive Controls
# =========================
allowed_countries = [
    'United States', 'USSR', 'Russia', 'China', 'United Kingdom',
    'Japan', 'France', 'India', 'Germany', 'Canada',
    'Israel', 'Italy', 'Brazil', 'South Korea', 'Iran',
    'North Korea', 'Australia', 'UAE', 'Argentina',
    'Mexico', 'Turkey', 'Indonesia'
]

# Replace USSR and Russia with USSR/Russia for grouping
allowed_groups = [c if c not in ['USSR', 'Russia'] else 'USSR/Russia' for c in allowed_countries]
allowed_groups = sorted(set(allowed_groups))

mission_options = ['All'] + sorted(df_payloads['MissionType'].dropna().unique().tolist())
country_options = ['All'] + allowed_groups
valid_statuses = ['All', 'Active', 'Defunct', 'Decayed']

min_year = int(df_payloads['Year'].min())
max_year = int(df_payloads['Year'].max())

mission_dropdown = widgets.Dropdown(options=mission_options, value='All', description='Mission:')
country_dropdown = widgets.Dropdown(options=country_options, value='All', description='Country:')

# Single-year slider
year_slider = widgets.IntSlider(
    value=2020,
    min=min_year,
    max=max_year,
    step=1,
    description='Up to Year:',
    continuous_update=False
)

status_checkboxes = widgets.SelectMultiple(
    options=valid_statuses,
    value=('All',),  # Default to "All" selected
    description='Status:',
    layout=widgets.Layout(width='30%'),
)

# =========================
#  2) Helper Function
# =========================
def compute_yearly_mission(df):
    """Pivot table of (Year, MissionType) -> count."""
    pivot = df.groupby(['Year', 'MissionType']).size().unstack(fill_value=0)
    return pivot.sort_index()

# =========================
#  3) Dashboard Function
# =========================
def merged_dashboard(mission, country, up_to_year, selected_statuses):
    clear_output(wait=True)

    # Start with a copy of all data
    df_filtered = df_payloads.copy()

    # Filter by mission
    if mission != 'All':
        df_filtered = df_filtered[df_filtered['MissionType'] == mission]

    # Filter by country
    if country != 'All':
        df_filtered = df_filtered[df_filtered['CountryGroup'] == country]

    # Filter up to the selected year
    df_filtered = df_filtered[df_filtered['Year'] <= up_to_year]

    if df_filtered.empty:
        print("No data available for selected filters.")
        return

    # ========== (A) AREA CHART: Mission Type Over Time ==========
    mission_pivot = compute_yearly_mission(df_filtered)
    figA, axA = plt.subplots(figsize=(10, 5))
    mission_pivot.plot(kind='area', stacked=True, ax=axA)
    axA.set_title("Satellite Launches Over Time by Mission Type", fontsize=14)
    axA.set_xlabel("Year")
    axA.set_ylabel("Satellites")
    plt.tight_layout()
    plt.show()

    # ========== (B) SPLIT FIGURE: Donut Chart + Operational Status ==========
    figB, (axB1, axB2) = plt.subplots(ncols=2, figsize=(14, 5))

    # ---------- (B1) DONUT CHART: SHARE OF SATELLITES BY TOP 5 COUNTRIES ----------
# ---------- (B1) DONUT CHART: SHARE OF SATELLITES BY TOP 5 COUNTRIES ----------
    top_countries = df_filtered['CountryGroup'].value_counts().nlargest(5)

    if top_countries.empty:
     axB1.text(0.5, 0.5, "No country data", ha='center')
    else:
     values = top_countries.values
     labels = top_countries.index
     total = values.sum()
     percentages = [f"{(v/total)*100:.1f}%" for v in values]

    # Create donut chart without internal labels
     wedges, texts = axB1.pie(
        values,
        startangle=140,
        radius=1.0,
        wedgeprops=dict(width=0.4, edgecolor='w')
     )

    # Create custom legend entries
    legend_labels = [f"{label}: {pct}" for label, pct in zip(labels, percentages)]
    axB1.legend(wedges, legend_labels, title="Country Share", loc="center right", bbox_to_anchor=(1.05, 0.5))

    axB1.set_title(f"Top 5 Countries (Up to Year {up_to_year})")
    axB1.axis('equal')  # Ensures the pie chart is circular


    # ---------- (B2) OPERATIONAL STATUS OVER TIME (GROUPED BY DECADE) ----------
    if 'All' in selected_statuses:
        df_status = df_filtered[df_filtered['OperationalStatus'].notna()].copy()
    else:
        df_status = df_filtered[df_filtered['OperationalStatus'].isin(selected_statuses)].copy()

    df_status['Decade'] = (df_status['Year'] // 10) * 10
    status_pivot = df_status.groupby(['Decade', 'OperationalStatus']).size().unstack(fill_value=0).sort_index()

    if not status_pivot.empty:
        status_pivot.plot(kind='bar', stacked=True, ax=axB2, colormap='tab20')
        axB2.set_title("Operational Status Over Time (by Decade)")
        axB2.set_xlabel("Decade")
        axB2.set_ylabel("Satellites")
        axB2.tick_params(axis='x', rotation=45)
    else:
        axB2.text(0.5, 0.5, "No status data", ha='center')

    plt.tight_layout()
    plt.show()

# =========================
#  4) Hook Up Widgets
# =========================
ui = widgets.VBox([
    widgets.HBox([mission_dropdown, country_dropdown]),
    widgets.HBox([year_slider, status_checkboxes])
])

out = widgets.interactive_output(
    merged_dashboard,
    {
        'mission': mission_dropdown,
        'country': country_dropdown,
        'up_to_year': year_slider,
        'selected_statuses': status_checkboxes
    }
)

display(ui, out)


Output()

In [21]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use('ggplot')

# =======================
# Setup & Preprocessing
# =======================
status_mapping = {
    'O': 'Active', 'D': 'Decayed', 'DK': 'Decayed', 'DSA': 'Decayed', 'DSO': 'Decayed',
    'N': 'Defunct', 'R': 'Defunct', 'R?': 'Defunct'
}
df_payloads['OperationalStatus'] = df_payloads['Status'].map(status_mapping)

allowed_countries = [
    'United States', 'USSR', 'Russia', 'China', 'United Kingdom',
    'Japan', 'France', 'India', 'Germany', 'Canada', 'Israel', 'Italy', 'Brazil',
    'South Korea', 'Iran', 'North Korea', 'Australia', 'UAE', 'Argentina',
    'Mexico', 'Turkey', 'Indonesia'
]
allowed_groups = [c if c not in ['USSR', 'Russia'] else 'USSR/Russia' for c in allowed_countries]
allowed_groups = sorted(set(allowed_groups))

mission_options = ['All'] + sorted(df_payloads['MissionType'].dropna().unique().tolist())
country_options = ['All'] + allowed_groups
valid_statuses = ['All', 'Active', 'Defunct', 'Decayed']

min_year = int(df_payloads['Year'].min())
max_year = int(df_payloads['Year'].max())

# =======================
# Widgets
# =======================
mission_dropdown = widgets.Dropdown(options=mission_options, value='All', description='Mission:')
country_dropdown = widgets.Dropdown(options=country_options, value='All', description='Country:')
year_slider = widgets.IntSlider(value=2020, min=min_year, max=max_year, step=1, description='Up to Year:')
status_checkboxes = widgets.SelectMultiple(
    options=valid_statuses,
    value=('All',),
    description='Status:',
    layout=widgets.Layout(width='100%')
)

# Style controls box with border and padding
controls_box = widgets.VBox(
    [
        widgets.HTML("<b>🔧 Filter Options</b>"),
        mission_dropdown,
        country_dropdown,
        year_slider,
        status_checkboxes
    ],
    layout=widgets.Layout(
        border='2px solid gray',
        padding='10px',
        width='100%',
        height='100%',
        background_color='#f9f9f9'
    )
)

# Output widget to render the dashboard
plot_output = widgets.Output()


# =======================
# Helpers
# =======================
def compute_yearly_mission(df):
    return df.groupby(['Year', 'MissionType']).size().unstack(fill_value=0).sort_index()

def merged_dashboard(mission, country, up_to_year, selected_statuses):
    with plot_output:
        clear_output(wait=True)

        df_filtered = df_payloads.copy()
        if mission != 'All':
            df_filtered = df_filtered[df_filtered['MissionType'] == mission]
        if country != 'All':
            df_filtered = df_filtered[df_filtered['CountryGroup'] == country]
        df_filtered = df_filtered[df_filtered['Year'] <= up_to_year]

        if df_filtered.empty:
            print("No data available for selected filters.")
            return

        fig = plt.figure(constrained_layout=True, figsize=(14, 10))
        spec = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)

        # Top-right: Area chart
        ax_area = fig.add_subplot(spec[0, :])
        mission_pivot = compute_yearly_mission(df_filtered)
        mission_pivot.plot(kind='area', stacked=True, ax=ax_area)
        ax_area.set_title("Satellite Launches Over Time by Mission Type")
        ax_area.set_xlabel("Year")
        ax_area.set_ylabel("Satellites")

        # Bottom-left: Donut chart
        ax_donut = fig.add_subplot(spec[1, 0])
        top_countries = df_filtered['CountryGroup'].value_counts().nlargest(5)
        if top_countries.empty:
            ax_donut.text(0.5, 0.5, "No country data", ha='center')
        else:
            values = top_countries.values
            labels = top_countries.index
            total = values.sum()
            percentages = [f"{(v / total) * 100:.1f}%" for v in values]
            wedges, _ = ax_donut.pie(values, startangle=140, radius=1.0, wedgeprops=dict(width=0.4, edgecolor='w'))
            legend_labels = [f"{label}: {pct}" for label, pct in zip(labels, percentages)]
            ax_donut.legend(wedges, legend_labels, title="Country Share", loc="center right", bbox_to_anchor=(1.05, 0.5))
            ax_donut.set_title(f"Top 5 Countries (Up to Year {up_to_year})")
            ax_donut.axis('equal')

        # Bottom-right: Operational status
        ax_status = fig.add_subplot(spec[1, 1])
        if 'All' in selected_statuses:
            df_status = df_filtered[df_filtered['OperationalStatus'].notna()].copy()
        else:
            df_status = df_filtered[df_filtered['OperationalStatus'].isin(selected_statuses)].copy()
        df_status['Decade'] = (df_status['Year'] // 10) * 10
        status_pivot = df_status.groupby(['Decade', 'OperationalStatus']).size().unstack(fill_value=0).sort_index()

        if not status_pivot.empty:
            status_pivot.plot(kind='bar', stacked=True, ax=ax_status, colormap='tab20')
            ax_status.set_title("Operational Status Over Time (by Decade)")
            ax_status.set_xlabel("Decade")
            ax_status.set_ylabel("Satellites")
            ax_status.tick_params(axis='x', rotation=45)
        else:
            ax_status.text(0.5, 0.5, "No status data", ha='center')

        fig.suptitle("Artificial Satellite Dashboard", fontsize=16, weight='bold')
        plt.show()

# =======================
# Link Controls → Dashboard
# =======================
def refresh_dashboard(_=None):
    merged_dashboard(
        mission_dropdown.value,
        country_dropdown.value,
        year_slider.value,from pathlib import Path

# Modify the code to use HTML-wrapped white background controls using ipywidgets+HTML
updated_white_controls_code = """
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use('ggplot')

# =======================
# Setup & Preprocessing
# =======================
status_mapping = {
    'O': 'Active', 'D': 'Decayed', 'DK': 'Decayed', 'DSA': 'Decayed', 'DSO': 'Decayed',
    'N': 'Defunct', 'R': 'Defunct', 'R?': 'Defunct'
}
df_payloads['OperationalStatus'] = df_payloads['Status'].map(status_mapping)

allowed_countries = [
    'United States', 'USSR', 'Russia', 'China', 'United Kingdom',
    'Japan', 'France', 'India', 'Germany', 'Canada', 'Israel', 'Italy', 'Brazil',
    'South Korea', 'Iran', 'North Korea', 'Australia', 'UAE', 'Argentina',
    'Mexico', 'Turkey', 'Indonesia'
]
allowed_groups = [c if c not in ['USSR', 'Russia'] else 'USSR/Russia' for c in allowed_countries]
allowed_groups = sorted(set(allowed_groups))

mission_options = ['All'] + sorted(df_payloads['MissionType'].dropna().unique().tolist())
country_options = ['All'] + allowed_groups
valid_statuses = ['All', 'Active', 'Defunct', 'Decayed']

min_year = int(df_payloads['Year'].min())
max_year = int(df_payloads['Year'].max())

# =======================
# Widgets
# =======================
mission_dropdown = widgets.Dropdown(options=mission_options, value='All', description='Mission:')
country_dropdown = widgets.Dropdown(options=country_options, value='All', description='Country:')
year_slider = widgets.IntSlider(value=2020, min=min_year, max=max_year, step=1, description='Up to Year:')
status_checkboxes = widgets.SelectMultiple(
    options=valid_statuses,
    value=('All',),
    description='Status:',
    layout=widgets.Layout(width='100%')
)

# HTML-styled white background control container
controls_box = widgets.HTML(
    value=\"\"\"
    <div style="background-color: white; padding: 15px; border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1); border: 1px solid #ddd;">
        <h3 style="margin-top: 0;">🔧 Filter Options</h3>
    </div>
    \"\"\")

controls_content = widgets.VBox([
    controls_box,
    mission_dropdown,
    country_dropdown,
    year_slider,
    status_checkboxes
])




# Save updated file
updated_path = Path("/mnt/data/dashboard_white_controls.py")
updated_path.write_text(updated_white_controls_code)

updated_path.name

        status_checkboxes.value
    )

mission_dropdown.observe(refresh_dashboard, names="value")
country_dropdown.observe(refresh_dashboard, names="value")
year_slider.observe(refresh_dashboard, names="value")
status_checkboxes.observe(refresh_dashboard, names="value")

# Initial Render
refresh_dashboard()

# =======================
# Display Layout
# =======================
final_layout = widgets.HBox([
    widgets.VBox([controls_box], layout=widgets.Layout(width='30%')),
    widgets.VBox([plot_output], layout=widgets.Layout(width='70%'))
])
display(final_layout)


In [29]:
from pathlib import Path


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use('ggplot')

# =======================
# Setup & Preprocessing
# =======================
status_mapping = {
    'O': 'Active', 'D': 'Decayed', 'DK': 'Decayed', 'DSA': 'Decayed', 'DSO': 'Decayed',
    'N': 'Defunct', 'R': 'Defunct', 'R?': 'Defunct'
}
df_payloads['OperationalStatus'] = df_payloads['Status'].map(status_mapping)

allowed_countries = [
    'United States', 'USSR', 'Russia', 'China', 'United Kingdom',
    'Japan', 'France', 'India', 'Germany', 'Canada', 'Israel', 'Italy', 'Brazil',
    'South Korea', 'Iran', 'North Korea', 'Australia', 'UAE', 'Argentina',
    'Mexico', 'Turkey', 'Indonesia'
]
allowed_groups = [c if c not in ['USSR', 'Russia'] else 'USSR/Russia' for c in allowed_countries]
allowed_groups = sorted(set(allowed_groups))

mission_options = ['All'] + sorted(df_payloads['MissionType'].dropna().unique().tolist())
country_options = ['All'] + allowed_groups
valid_statuses = ['All', 'Active', 'Defunct', 'Decayed']

min_year = int(df_payloads['Year'].min())
max_year = int(df_payloads['Year'].max())

# =======================
# Widgets
# =======================
mission_dropdown = widgets.Dropdown(options=mission_options, value='All', description='Mission:')
country_dropdown = widgets.Dropdown(options=country_options, value='All', description='Country:')
year_slider = widgets.IntSlider(value=2020, min=min_year, max=max_year, step=1, description='Up to Year:')
status_checkboxes = widgets.SelectMultiple(
    options=valid_statuses,
    value=('All',),
    description='Status:',
    layout=widgets.Layout(width='100%')
)


# Styled HTML header (white box)
controls_header = widgets.HTML(
    value="""
    <div style="background-color: white; padding: 15px; border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1); border: 1px solid #ddd;">
        <h3 style="margin-top: 0; color: #333;">🔧 Filter Options</h3>
    </div>
    """
)

# Assemble final control box
controls_box = widgets.VBox(
    [
        controls_header,
        mission_dropdown,
        country_dropdown,
        year_slider,
        status_checkboxes
    ],
    layout=widgets.Layout(
        padding='5px',
        width='100%'
    )
)

# Output widget to render the dashboard
plot_output = widgets.Output()

# =======================
# Helpers
# =======================
def compute_yearly_mission(df):
    return df.groupby(['Year', 'MissionType']).size().unstack(fill_value=0).sort_index()

def merged_dashboard(mission, country, up_to_year, selected_statuses):
    with plot_output:
        clear_output(wait=True)

        df_filtered = df_payloads.copy()
        if mission != 'All':
            df_filtered = df_filtered[df_filtered['MissionType'] == mission]
        if country != 'All':
            df_filtered = df_filtered[df_filtered['CountryGroup'] == country]
        df_filtered = df_filtered[df_filtered['Year'] <= up_to_year]

        if df_filtered.empty:
            print("No data available for selected filters.")
            return

        fig = plt.figure(constrained_layout=True, figsize=(14, 10))
        spec = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)

        ax_area = fig.add_subplot(spec[0, :])
        mission_pivot = compute_yearly_mission(df_filtered)
        mission_pivot.plot(kind='area', stacked=True, ax=ax_area)
        ax_area.set_title("Satellite Launches Over Time by Mission Type")
        ax_area.set_xlabel("Year")
        ax_area.set_ylabel("Satellites")

        ax_donut = fig.add_subplot(spec[1, 0])
        top_countries = df_filtered['CountryGroup'].value_counts().nlargest(5)
        if top_countries.empty:
            ax_donut.text(0.5, 0.5, "No country data", ha='center')
        else:
            values = top_countries.values
            labels = top_countries.index
            total = values.sum()
            percentages = [f"{(v / total) * 100:.1f}%" for v in values]
            wedges, _ = ax_donut.pie(values, startangle=140, radius=1.0, wedgeprops=dict(width=0.4, edgecolor='w'))
            legend_labels = [f"{label}: {pct}" for label, pct in zip(labels, percentages)]
            ax_donut.legend(wedges, legend_labels, title="Country Share", loc="center right", bbox_to_anchor=(1.05, 0.5))
            ax_donut.set_title(f"Top 5 Countries (Up to Year {up_to_year})")
            ax_donut.axis('equal')
            pos = ax_donut.get_position()
            ax_donut.set_position([pos.x0 - 0.05, pos.y0, pos.width, pos.height])

        ax_status = fig.add_subplot(spec[1, 1])
        if 'All' in selected_statuses:
            df_status = df_filtered[df_filtered['OperationalStatus'].notna()].copy()
        else:
            df_status = df_filtered[df_filtered['OperationalStatus'].isin(selected_statuses)].copy()
        df_status['Decade'] = (df_status['Year'] // 10) * 10
        status_pivot = df_status.groupby(['Decade', 'OperationalStatus']).size().unstack(fill_value=0).sort_index()

        if not status_pivot.empty:
            status_pivot.plot(kind='bar', stacked=True, ax=ax_status, colormap='tab20')
            ax_status.set_title("Operational Status Over Time (by Decade)")
            ax_status.set_xlabel("Decade")
            ax_status.set_ylabel("Satellites")
            ax_status.tick_params(axis='x', rotation=45)
        else:
            ax_status.text(0.5, 0.5, "No status data", ha='center')

        fig.suptitle("Artificial Satellite Dashboard", fontsize=16, weight='bold')
        plt.show()

# =======================
# Link Controls → Dashboard
# =======================
def refresh_dashboard(_=None):
    merged_dashboard(
        mission_dropdown.value,
        country_dropdown.value,
        year_slider.value,
        status_checkboxes.value
    )

mission_dropdown.observe(refresh_dashboard, names="value")
country_dropdown.observe(refresh_dashboard, names="value")
year_slider.observe(refresh_dashboard, names="value")
status_checkboxes.observe(refresh_dashboard, names="value")

# Initial Render
refresh_dashboard()

# =======================
# Display Layout
# =======================
final_layout = widgets.HBox([
    widgets.VBox([controls_box], layout=widgets.Layout(width='30%')),
    widgets.VBox([plot_output], layout=widgets.Layout(width='70%'))
])
display(final_layout)



In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Prepare data for interactive plot (reuse launch_pivot from above)
years = launch_pivot.index.values

# Widgets for country selection and year range
country_options = ["All"] + ["United States", "USSR/Russia", "China", "Other"]
country_sel = widgets.SelectMultiple(
    options=country_options,
    value=("All",),  # default show global total
    description="Country",
    disabled=False
)
year_range = widgets.IntRangeSlider(
    value=[int(years.min()), int(years.max())],
    min=int(years.min()),
    max=int(years.max()),
    step=1,
    description='Year Range',
    continuous_update=False
)

# Function to update the plot based on widget values
def update_launch_plot(country_tuple, year_range=year_range.value):
    sel_countries = list(country_tuple)
    fig, ax = plt.subplots(figsize=(7,4))
    start, end = year_range
    year_index = launch_pivot.index[(launch_pivot.index>=start) & (launch_pivot.index<=end)]
    data = launch_pivot.reindex(year_index).fillna(0)
    total = data.sum(axis=1)
    if "All" in sel_countries or len(sel_countries)==0:
        # Plot global total
        ax.plot(year_index, total, label="All Countries", color="#333333", linewidth=2)
    else:
        if "Other" in sel_countries:
            # compute others as total minus known three
            data['Other'] = total - data[["United States","USSR/Russia","China"]].sum(axis=1)
        for country in sel_countries:
            if country in data.columns:
                ax.plot(year_index, data[country], label=country, linewidth=2)
    ax.set_xlabel("Year")
    ax.set_ylabel("Satellites Launched")
    ax.set_title("Satellite Launches per Year")
    ax.legend(loc="upper left")
    plt.show()

# Link widgets and display
widgets.interactive(update_launch_plot, country_tuple=country_sel, year_range=year_range)


interactive(children=(SelectMultiple(description='Country', index=(0,), options=('All', 'United States', 'USSR…

In [ ]:
# Compute overall counts by orbit class and status (in orbit or reentered)
orbit_status_counts = df_payloads.groupby(['OrbitClass','InOrbit']).size().unstack(fill_value=0)
# Rename status columns for clarity
orbit_status_counts.columns = ['Reentered','In Orbit']

# Widgets for orbit class selection and status filter
orbit_options = sorted(orbit_status_counts.index.unique())
orbit_sel = widgets.SelectMultiple(
    options=orbit_options,
    value=tuple(orbit_options),  # default select all
    description="Orbits",
    disabled=False
)
status_filter = widgets.RadioButtons(
    options=['All', 'In Orbit', 'Reentered'],
    value='All',
    description='Status',
    disabled=False
)

def update_orbit_plot(orbits, status):
    selected = list(orbits)
    data = orbit_status_counts.reindex(index=selected, fill_value=0)
    # Apply status filter
    if status == 'In Orbit':
        data_plot = data[['In Orbit']]
    elif status == 'Reentered':
        data_plot = data[['Reentered']]
    else:
        # All: plot stacked both
        data_plot = data

    fig, ax = plt.subplots(figsize=(6,4))
    if status == 'All':
        # Stacked bars: reentered at bottom, in orbit on top
        ax.bar(data_plot.index, data_plot['Reentered'], label='Reentered', color=status_colors['Reentered'])
        ax.bar(data_plot.index, data_plot['In Orbit'], bottom=data_plot['Reentered'], label='Still In Orbit', color=status_colors['In Orbit'])
    else:
        # Single bars for either In Orbit or Reentered
        color = status_colors['In Orbit'] if status=='In Orbit' else status_colors['Reentered']
        ax.bar(data_plot.index, data_plot[status], color=color)
    ax.set_title("Satellites by Orbit and Status")
    ax.set_xlabel("Orbit Class")
    ax.set_ylabel("Number of Satellites")
    if status == 'All':
        ax.legend(title="Status")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

widgets.interactive(update_orbit_plot, orbits=orbit_sel, status=status_filter)


interactive(children=(SelectMultiple(description='Orbits', index=(0, 1, 2, 3, 4, 5, 6), options=('Deep Space',…

In [ ]:
# Compute mission type counts by country
country_mission = df_payloads.groupby(['CountryGroup','MissionType']).size().unstack(fill_value=0)

country_options = sorted(country_mission.index.unique())
country_dropdown = widgets.Dropdown(
    options=country_options,
    value='United States',
    description='Country',
    disabled=False
)

def update_mission_bar(country):
    counts = country_mission.loc[country]
    # Only plot the three main categories and lump others
    main_cats = ['Communications','Military','Scientific']
    other_count = counts.drop(main_cats).sum()
    data = counts[main_cats].copy()
    data['Other'] = other_count
    fig, ax = plt.subplots(figsize=(6,4))
    bars = ax.bar(data.index, data.values, color=[category_colors.get(cat, "#7f7f7f") for cat in data.index])
    ax.set_title(f"Satellite Types - {country}")
    ax.set_ylabel("Number of Satellites")
    # Annotate counts on top of bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height+1, str(int(height)), ha='center', va='bottom', fontsize=8)
    plt.tight_layout()
    plt.show()

widgets.interactive(update_mission_bar, country=country_dropdown)


interactive(children=(Dropdown(description='Country', index=18, options=('AU', 'CSSR', 'Canada', 'China', 'E',…